In [5]:
# -*- coding: utf-8 -*-

import openai_api as api


text = "python是怎样执行的"

result = api.completions(system_prompt=None, context_input=None, input_prompt=text)
print(result)
print(result['choices'][0]['message']['content'])

{
  "id": "chatcmpl-86XpqJ3w5pKdCLBp88GKrDJBhc5pB",
  "object": "chat.completion",
  "created": 1696570722,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Python\u7684\u6267\u884c\u8fc7\u7a0b\u53ef\u4ee5\u5206\u4e3a\u4ee5\u4e0b\u51e0\u4e2a\u6b65\u9aa4\uff1a\n\n1. \u89e3\u91ca\u5668\u52a0\u8f7d\u6e90\u4ee3\u7801\uff1aPython\u89e3\u91ca\u5668\u4f1a\u8bfb\u53d6\u6e90\u4ee3\u7801\u6587\u4ef6\uff0c\u5e76\u6309\u7167\u4ece\u4e0a\u5230\u4e0b\u7684\u987a\u5e8f\u89e3\u6790\u4ee3\u7801\u3002\n\n2. \u8bcd\u6cd5\u5206\u6790\uff1a\u89e3\u91ca\u5668\u5c06\u6e90\u4ee3\u7801\u5206\u89e3\u6210\u4e00\u7cfb\u5217\u7684\u8bcd\u6cd5\u5143\u7d20\uff0c\u5982\u5173\u952e\u5b57\u3001\u6807\u8bc6\u7b26\u3001\u8fd0\u7b97\u7b26\u3001\u5e38\u91cf\u7b49\u3002\n\n3. \u8bed\u6cd5\u5206\u6790\uff1a\u89e3\u91ca\u5668\u6839\u636e\u8bcd\u6cd5\u5143\u7d20\u6784\u5efa\u8bed\u6cd5\u6811\uff0c\u8868\u793a\u7a0b\u5e8f\u7684\u8bed\

In [6]:
# 非流式返回，等待回答结束后返回
# imports
import openai  # for OpenAI API calls
import time  # for measuring time duration of API calls
import openai_api as api

# Example of an OpenAI ChatCompletion request
# https://platform.openai.com/docs/guides/chat

# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    temperature=0,
)

# calculate the time it took to receive the response
response_time = time.time() - start_time

# print the time delay and text received
print(f"Full response received {response_time:.2f} seconds after request")
print(f"Full response received:\n{response}")

reply = response['choices'][0]['message']
print(f"Extracted reply: \n{reply}")

reply_content = response['choices'][0]['message']['content']
print(f"Extracted content: \n{reply_content}")



Full response received 7.45 seconds after request
Full response received:
{
  "id": "chatcmpl-86Xpwg2px7TxCTFCTcqDEW4khyFVv",
  "object": "chat.completion",
  "created": 1696570728,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 36,
    "completion_tokens": 299,
    "total_tokens": 335
  }
}
Extracted reply: 
{
  "role": "assistant",
  "content": "1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2

In [7]:
# 流式返回
import openai  # for OpenAI API calls
import time  # for measuring time duration of API calls
import openai_api as api

# Example of an OpenAI ChatCompletion request
# https://platform.openai.com/docs/guides/chat

# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/guides/chat

# a ChatCompletion request
# response = openai.ChatCompletion.create(
#     model='gpt-3.5-turbo',
#     messages=[
#         {'role': 'user', 'content': "What's 1+1? Answer in one word."}
#     ],
#     temperature=0,
#     stream=True  # this time, we set stream=True
# )

# for chunk in response:
#     print(chunk)

# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/guides/chat

# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    temperature=0,
    stream=True  # again, we set stream=True
)

# create variables to collect the stream of chunks
collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in response:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk['choices'][0]['delta']  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text

# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")
full_reply_content = ''.join([m.get('content', '') for m in collected_messages])
print(f"Full conversation received: {full_reply_content}")


Message received 0.37 seconds after request: {
  "role": "assistant",
  "content": ""
}
Message received 0.37 seconds after request: {
  "content": "1"
}
Message received 0.39 seconds after request: {
  "content": ","
}
Message received 0.42 seconds after request: {
  "content": " "
}
Message received 0.44 seconds after request: {
  "content": "2"
}
Message received 0.49 seconds after request: {
  "content": ","
}
Message received 0.50 seconds after request: {
  "content": " "
}
Message received 0.51 seconds after request: {
  "content": "3"
}
Message received 0.53 seconds after request: {
  "content": ","
}
Message received 0.55 seconds after request: {
  "content": " "
}
Message received 0.58 seconds after request: {
  "content": "4"
}
Message received 0.60 seconds after request: {
  "content": ","
}
Message received 0.62 seconds after request: {
  "content": " "
}
Message received 0.64 seconds after request: {
  "content": "5"
}
Message received 0.67 seconds after request: {
  "cont